In [1]:
import pandas as pd
import numpy as np
import string
import re
import nltk
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [2]:
df = pd.read_csv('data\dataset_tweet_sentiment_opini_film.csv')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\MyBook PRO K7V\AppData\Local\Temp\ipykernel_13700\228168114.py:1: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv('data\dataset_tweet_sentiment_opini_film.csv')


In [3]:
df = df.drop('Id', axis='columns')

In [4]:
def preprocass(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    return text

In [5]:
def tokenize(text):
    return text.lower().split()

In [6]:
factory = StopWordRemoverFactory()
stopoword = factory.get_stop_words()
def remove_stopwords(tokens):
    cleaning_text = [token for token in tokens if token not in stopoword]
    return cleaning_text

In [7]:
stem = StemmerFactory()
stemmer = stem.create_stemmer()
def stem_token(tokens):
    stemmid_tokens = [stemmer.stem(token) for token in tokens]
    return stemmid_tokens

In [8]:
df['clean text'] = df['Text Tweet'].apply(preprocass)
df['tokens'] = df['clean text'].apply(tokenize)
df['stop'] = df['tokens'].apply(remove_stopwords)
df['stemmid_tokens'] = df['stop'].apply(stem_token)

In [ ]:
df.to_csv('data/data clean', index=False)

In [9]:
corpus = df['stemmid_tokens'].apply(lambda x: ' '.join(x))

In [10]:
def compute_idf(word, corpus):
    N = len(corpus)
    count = sum(1 for tokens in corpus if word in tokens)
    if count == 0:
        return 0
    else :
        return np.log(N/count + 1)

In [11]:
vocab = set(word for text in corpus for word in text)
idfs = {word : compute_idf(word, corpus)for word in vocab}

In [12]:
def corpus_tfidf(text, idfs):
    tfidf = {}
    total_word = len(text)
    for word in set(text):
        tf = text.count(word)/total_word
        tfidf[word] = tf*idfs.get(word, 0)
    return tfidf

In [13]:
print(df['stemmid_tokens'].apply(type))

print(df['stemmid_tokens'].isnull().sum())

0      <class 'list'>
1      <class 'list'>
2      <class 'list'>
3      <class 'list'>
4      <class 'list'>
            ...      
195    <class 'list'>
196    <class 'list'>
197    <class 'list'>
198    <class 'list'>
199    <class 'list'>
Name: stemmid_tokens, Length: 200, dtype: object
0


In [14]:
tfidf_text = [compute_idf(text, idfs) for text in corpus]

In [15]:
def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    return dot_product / (norm_v1 * norm_v2)

In [16]:
def create_features_vector(text, vocabulary):
    vector = [1 if word in text else 0 for word in vocabulary]
    return np.array(vector)

In [21]:
def knn_classify(k, sample_vector, data, labels, vocabulary):
    similarities = []
    for i, row in data.iterrows():
        text_tokens = row['Sentiment'].split()
        text_vector = create_features_vector(text_tokens, vocabulary)
        similarity  = cosine_similarity(sample_vector, text_vector)
        similarities.append((similarity, labels[i]))
    similarities.sort(reverse=True)
    k_nearest = similarities[:k]
    counts = {}
    for _, label in k_nearest:
        counts[label] = counts.get(label, 0) + 1
    majority_label = max(counts, key = counts.get)
    return majority_label

In [18]:
corpuss = ' '.join(corpus).split()
vocabulary = set(corpuss)

In [19]:
sample_text = "saya suka olahraga"
sample_tokens = stem_token(tokenize(preprocass(sample_text)))
sample_vector = create_features_vector(sample_tokens, vocabulary)

In [22]:
k = 3
predicted_label = knn_classify(k, sample_vector, df, df['Sentiment'], vocabulary)
print("predicted_label :", predicted_label)

predicted_label : negative


C:\Users\MyBook PRO K7V\AppData\Local\Temp\ipykernel_13700\1871220151.py:5: RuntimeWarning: invalid value encountered in scalar divide
  return dot_product / (norm_v1 * norm_v2)
